In [1]:
import astropy.units as u
import numpy as np
import pandas as pd
import xarray

data_df = pd.read_csv("/home/dominik/HoulbergNSTX.csv", index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '/home/dominik/HoulbergNSTX.csv'

In [ ]:
data_df[["x", "ToverT0", "ToverT0_derivative"]].plot(x="x")

In [ ]:
data_df[["x", "n_D", "n_C", "n_e"]].plot(x="x")

In [ ]:
data_df[["x", "n_D_derivative", "n_C_derivative", "n_e_derivative"]].plot(x="x")

In [ ]:
T_i = T_e = T_C6 = data_df["ToverT0"] * 0.5 * 1000
dT_i = dT_e = dT_C6 = data_df["ToverT0_derivative"] * 0.5 * 1000
n_e, dn_e = data_df["n_e"], data_df["n_e_derivative"]
n_i, dn_i = data_df["n_D"], data_df["n_D_derivative"]
n_C6, dn_C6 = data_df["n_C"], data_df["n_C_derivative"]

rho = data_df.x.values

## Multiple flux surfaces - radial grid

In [ ]:
from plasmaboundaries import NSTX_single_null

NSTX_single_null

NSTX_Bt0 = 0.3 * u.T
NSTX_R0 = 0.8 * u.m
NSTX_a0 = 0.64 * u.m
NSTX_I = 1 * u.MA
from plasmapy.plasma.symbolicequilibrium import SymbolicEquilibrium

params = {"aspect_ratio": 1.25, "A": -0.05, "elongation": 2, "triangularity": 0.3}
# TODO this is still not taken in

eq = SymbolicEquilibrium(
    **NSTX_single_null,
    B0=NSTX_Bt0.si.value,  # TODO handle quantity input
    config="single-null",
)
rminmaxstep = (
    0.1,
    1.9,
    0.001,
)  # these definitely, unfortunately, need to be moved into SymbolicEquilibrium
zminmaxstep = (-2, 2, 0.001)
eq.plot(rminmaxstep=rminmaxstep, zminmaxstep=zminmaxstep);

In [ ]:
from tqdm.auto import tqdm

In [ ]:
surfaces = list(
    tqdm(
        eq.get_multiple_flux_surfaces(
            rho_values=rho, rminmaxstep=rminmaxstep, zminmaxstep=zminmaxstep
        ),
        total=len(rho),
    )
)

ax = eq.plot(rminmaxstep=rminmaxstep, zminmaxstep=zminmaxstep)
for psi, fs in surfaces:
    if fs is not None:
        fs.plot(ax)
ax.legend().remove()
plt.tight_layout()

## `FlowCalculator`

In [ ]:
rho_to_surface = {key: value for key, value in surfaces};

In [ ]:
import xarray

dataset_H1 = xarray.Dataset(
    {
        "T": ("rho", T_i),
        "gradT": ("rho", dT_i),
        "n": ("rho", n_i),
        "gradn": ("rho", dn_i),
    },
    coords={"rho": rho, "particle": "H 1+"},
    attrs={
        "T unit": u.eV,
        "n unit": u.m ** -3,
        "gradT unit": u.eV / u.m,
        "gradn unit": u.m ** -3 / u.m,
    },
)

dataset_C6 = xarray.Dataset(
    {
        "T": ("rho", T_C6),
        "gradT": ("rho", dT_C6),
        "n": ("rho", n_C6),
        "gradn": ("rho", dn_C6),
    },
    coords={"rho": rho, "particle": "C 6+"},
    attrs={
        "T unit": u.eV,
        "n unit": u.m ** -3,
        "gradT unit": u.eV / u.m,
        "gradn unit": u.m ** -3 / u.m,
    },
)
dataset_e = xarray.Dataset(
    {
        "T": ("rho", T_e),
        "gradT": ("rho", dT_e),
        "n": ("rho", n_e),
        "gradn": ("rho", dn_e),
    },
    coords={"rho": rho, "particle": "e-"},
    attrs={
        "T unit": u.eV,
        "n unit": u.m ** -3,
        "gradT unit": u.eV / u.m,
        "gradn unit": u.m ** -3 / u.m,
    },
)

dataset = xarray.concat([dataset_H1, dataset_C6, dataset_e], dim="particle")
# dataset["rho"] = ("rho", rho)
dataset["charges"] = ("particle", [1, 6, -1])
dataset["charge_density"] = "rho", (dataset.charges * dataset.n).sum(dim="particle")
dataset

In [ ]:
dataset["psi"] = eq.rho_to_psi(rho)
dataset

In [ ]:
from tqdm import auto as tqdm

from plasmapy.transport.flows import FlowCalculator

fcs = []
import warnings

for i, (ψ, surface) in enumerate(tqdm.tqdm(surfaces)):
    if surface is None:
        print(f"Skipping {i}-th surface at {ψ}")
        continue
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            fcs.append(FlowCalculator.from_xarray_surface(dataset.isel(rho=i), surface))
    except ImportError as e:
        display(e)

In [ ]:
datasets = [fc.to_dataset() for fc in tqdm.tqdm(fcs)];

In [ ]:
results = xarray.concat(datasets, dim="rho")
scaling = (fcs[0].bootstrap_current.unit / NSTX_Bt0).to(u.MA / u.m ** 2)
results = results.assign(
    bootstrap_current_normalized=results.bootstrap_current * scaling
)

In [ ]:
results.bootstrap_current_normalized.plot.line()

In [ ]:
results.thermal_conductivity

In [ ]:
results.diffusion_coefficient.sel(particle="C 6+")

In [ ]:
results.diffusion_coefficient.plot.line(x="rho")
plt.xlim(0, 0.8)
plt.ylim(-10, 10)

In [ ]:
results.thermal_conductivity.plot.line(x="rho")

In [ ]:
results.total_particle_flux.plot.line(x="rho")

In [ ]:
results.CL_particle_flux.plot.line(x="rho")

In [ ]:
results.BP_particle_flux.plot.line(x="rho")

In [ ]:
results.PS_particle_flux.plot.line(x="rho")

In [ ]:
import pandas as pd

df = pd.read_csv("/home/dominik/Inbox/NSTXplot1.csv")

df.plot.line(x="x")
results.bootstrap_current_normalized.plot.line(x="rho", label="My bootstrap current")
plt.legend()

In [ ]:
directions = ["poloidal", "toroidal", "parallel", "perpendicular"]
for index, label in enumerate(directions):
    ax = eq.plot(rminmaxstep=rminmaxstep, zminmaxstep=zminmaxstep)
    surfaces[5][1].plot(
        ax, quantity=fcs[-5].local_flow_velocities["H 1+"][index].value, legend=False
    )
    ax.set_title(label)

In [ ]:
for index, label in enumerate(directions):
    ax = eq.plot(rminmaxstep=rminmaxstep, zminmaxstep=zminmaxstep)
    for (psi, surface), fc in zip(surfaces, fcs):
        surface.plot(
            ax,
            quantity=fc.local_flow_velocities["H 1+"][index].value,
            legend=False,
            colorbar=False,
        )
    ax.set_title(label)